# Configure program environment of dh

## Install PySCF

- In general case, install PySCF from pypi should work in most cases.

- A more step is add a statement in `pyscf/__init__.py`:
    
    ```python
    '''
        >>> from pyscf import gto, scf
        >>> mol = gto.M(atom='H 0 0 0; H 0 0 1.2', basis='cc-pvdz')
        >>> mol.apply(scf.RHF).run()
        converged SCF energy = -1.06111199785749
        -1.06111199786

    '''
    __path__ = __import__('pkgutil').extend_path(__path__, __name__)  # <<<< ADD THIS LINE
    __version__ = '2.1.1'

    import os
    import sys
    # Avoid too many threads being created in OMP loops.
    # See issue https://github.com/pyscf/pyscf/issues/317
    ```

- For advanced usage (such as using range-separate doubly hybrids such as RS-PBE-P86), one may use libxc version to the master branch of libxc (instead of stable version 6.x). You may compile the PySCF for yourself by modifing `pyscf/lib/CMakeLists.txt` and using cmake to build library of pyscf.

    - For some cases, if using certain version of mkl (maybe 2023.0), you may encounter library loading problems when importing pyscf. A possible solution is declaring the following variables in your shell environment, for example bash:
    
    ```bash
    # extracted from /apps/dh/qscf.env on group server
    export CONDA_PATH=/share/apps/dh/miniconda3/envs/pyscf
    export LD_PRELOAD=$LD_PRELOAD:$CONDA_PATH/lib/libmkl_intel_lp64.so
    export LD_PRELOAD=$LD_PRELOAD:$CONDA_PATH/lib/libmkl_core.so
    export LD_PRELOAD=$LD_PRELOAD:$CONDA_PATH/lib/libiomp5.so
    export LD_PRELOAD=$LD_PRELOAD:$CONDA_PATH/lib/libmkl_intel_thread.so
    ```

## Install pyscf-forge

For downloading and git workflow, 

- Git clone pyscf-forge from gitee `https://gitee.com/ajz34/pyscf-forge`.
- **IMPORTANT** change branch to `pre-0.2` (currently we develop this project from this branch).
- **IMPORTANT** create branch from `pre-0.2` by yourself.
- Create pull-request on gitee, and merge to branch `ajz34/pyscf-forge:pre-0.2`.
    - By finalizing a minor version 0.2, ajz34 may create a pull-request to github `pyscf/pyscf-forge:dh-0.2`. All contributions from contributors should be available in git log and will be merged into original repo pyscf/pyscf-forge (if PR accepted).
- **IMPORTANT** if you are not familiar to git, please make a backup before conducting any operations other than `git add` and `git commit`. 

To make pyscf-forge available,

- Since we only use and develop `dh`, then just make pyscf-forge as a usual extension.
    
    ```bash
    export PYSCF_EXT_PATH=path_to_pyscf_forge:$PYSCF_EXT_PATH
    ```

- Finally, try out if importing successful.

In [1]:
from pyscf import gto, dh
mol = gto.Mole(atom="O; H 1 0.94; H 1 0.94 2 104.5", basis="cc-pVTZ").build()
mf = dh.DH(mol, xc="XYG3").build_scf(route_scf="RI-J").run(frozen="FreezeNobleGasCore")
mf.e_tot

converged SCF energy = -76.458949560193
[RESULT] Energy of exchange 0.8033*HF:      -7.198335997044
[RESULT] Energy of process_energy_low_rung (non_exx): -2.0158615676050706
[RESULT] Energy of process_energy_low_rung (total): -9.21419756464918
[RESULT] Energy corr MP2 of same-spin:      -0.0867173183
[RESULT] Energy corr MP2 of oppo-spin:      -0.2654861318
[RESULT] Energy corr MP2 of total:          -0.3522034500
[RESULT] Energy of correlation MP2(0.3211, 0.3211):      -0.113092527809
[RESULT] Energy of 0.8033*HF + 0.2107*B88 - 0.014*LDA, 0.6789*LYP + MP2(0.3211, 0.3211):     -76.417238524599


-76.41723852459926

- Reproduce this result with code by PySCF:

In [2]:
from pyscf import dft, df
from pyscf.mp.dfmp2 import DFMP2
mf_s = dft.RKS(mol, xc="B3LYPg").density_fit(only_dfj=True).run()
mf_n = dft.RKS(mol, xc="0.8033*HF + 0.2107*B88 - 0.014*LDA, 0.6789*LYP").density_fit(only_dfj=True)
e_low_rung = mf_n.energy_tot(dm=mf_s.make_rdm1())
mf_mp = DFMP2(mf_s)
mf_mp.with_df = df.DF(mol, auxbasis="cc-pVTZ-ri")
mf_mp.run(frozen=[0])
e_corr_mp2 = 0.3211 * mf_mp.e_corr
e_tot = e_low_rung + e_corr_mp2
e_tot

converged SCF energy = -76.458949560193
E(DFMP2) = -76.8111530102356  E_corr = -0.35220345004259
E(SCS-DFMP2) = -76.8064386910706  E_corr = -0.347489130877568


-76.41723852459097